In [1]:
from collections import Counter
from pymongo import MongoClient
import pandas as pd
import numpy as np
import keras
import re

/Users/clementtailleur/Documents/BetIt/Py3_BetIT/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def _connect_mongo(host, port, db):
    """ A util for making a connection to mongo """
    conn = MongoClient(host, port)
    return conn[db]


def read_mongo(db, collection, query={}, host='localhost', port=27017, username=None, password=None, no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

df = read_mongo(db="WorldCup", collection="Tweets", no_id=True)
df.head()

,created_at,favorite_count,location,retweeted,retweeted_count,text
0,2018-05-26 13:20:00,0,None,False,0,RT @LFC: When you turn around and spot @Alex_O...
1,2018-05-26 13:20:01,0,JHB|FREE STATE|NORTHERN CAPE|NORTH WEST,False,0,Real Madrid or Liverpool? 🏆\nRonaldo or M.Sala...
2,2018-05-26 13:20:00,0,UK & Ireland,False,0,Massive #UCLfinal offer! ⚽️\n\nBack #RMFC at 4...
3,2018-05-26 13:20:02,0,Bolton,False,0,RT @paddypower: Ahead of the Champions League ...
4,2018-05-26 13:20:02,0,None,False,0,RT @ninanaka: Hope it will be an unforgettable...


In [3]:
df.drop_duplicates(subset="text", keep='first', inplace=True)
df.shape

(184310, 6)

In [4]:
%%time
HASHTAG = "#UCLfinal"
def clean_tweets(text):
    text_clean = re.sub(r"^RT \@(.*)\:", "", text)
    text_clean = re.sub(HASHTAG, "", text_clean)
    text_clean = text_clean.lower()
    text_clean = re.sub(r"(?m)^\s+", "", text_clean)
    text_clean = re.sub("\n", " ", text_clean)
    text_clean = re.sub(r"""(?:(?:https?|ftp):\/\/|\b(?:[a-z\d]+\.))(?:(?:[^\s()<>]+|\((?:[^\s()<>]+|(?:\([^\s()<>]+\)))?\))+(?:\((?:[^\s()<>]+|(?:\(?:[^\s()<>]+\)))?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))?""", "", text_clean)
    text_clean = re.sub("//", "", text_clean)
    return text_clean

df["tweets_cleaned"] = df["text"].apply(lambda x: clean_tweets(x))

CPU times: user 18min 1s, sys: 3.34 s, total: 18min 5s
Wall time: 18min 11s


In [5]:
df_not_empty = df[df["tweets_cleaned"] != ""].copy()
df_not_empty.shape

(176952, 7)

In [6]:
df_not_empty["len_tweet_cleaned"] = df_not_empty["tweets_cleaned"].apply(lambda x: len(x.split()))
df_not_empty.head()

,created_at,favorite_count,location,retweeted,retweeted_count,text,tweets_cleaned,len_tweet_cleaned
0,2018-05-26 13:20:00,0,None,False,0,RT @LFC: When you turn around and spot @Alex_O...,when you turn around and spot @alex_oxchambo a...,12
1,2018-05-26 13:20:01,0,JHB|FREE STATE|NORTHERN CAPE|NORTH WEST,False,0,Real Madrid or Liverpool? 🏆\nRonaldo or M.Sala...,real madrid or liverpool? 🏆 ronaldo or #uclfi...,10
2,2018-05-26 13:20:00,0,UK & Ireland,False,0,Massive #UCLfinal offer! ⚽️\n\nBack #RMFC at 4...,massive offer! ⚽️ back #rmfc at 40/1 or #lfc ...,14
4,2018-05-26 13:20:02,0,None,False,0,RT @ninanaka: Hope it will be an unforgettable...,hope it will be an unforgettable journey for t...,16
5,2018-05-26 13:20:02,0,"Verulam, South Africa",False,0,"RT @MelissaReddy_: ""Pick any continent, any co...","""pick any continent, any country, any city and...",20


In [7]:
df_valid = df_not_empty[df_not_empty["len_tweet_cleaned"] > 3]
df_valid.shape

(155700, 8)

In [8]:
def flat_list(l):
    list_final = []
    for i in l:
        if type(i) is list:
            list_final.extend(i)
        else:
            list_final.append(i)
    return list_final

def TM_SC_create_words(list_text, nb_min_word = 2):
    list_list_words = [re.findall(r"\w+",x) for x in list_text]
    WORDS = Counter(flat_list(list_list_words))
    WORDS_to_correct = {k:v for (k,v) in WORDS.items() if v <nb_min_word} #Dictionnaire ajouter key value
    WORDS = {k:v for (k,v) in WORDS.items() if v >=nb_min_word}
    return WORDS, WORDS_to_correct



WORDS, WORDS_to_correct = TM_SC_create_words(df_valid["tweets_cleaned"].values)
s = 0
for v in WORDS.values(): s += v
print("Nombre de mots:", s)

Nombre de mots: 1711574
